#if remote_unit_id and timestamp are the same, then find the total sum of entries and exits

### Creating busyness percentage for each station
- For each remote unit id, get a sum of the abs value of the difference in entries and exits
- The busyness of a place is how many people are currently there, so exits-entries along with the magnitude in the sum of both values, can do both
- Sum of entries and exits for each remote unit id at that time
- Exits - entries for each remote unit id at that time
- Normalize each of these values?

In [ ]:
'''df_turn['day'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.day_name()
df_turn['date'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.date
df_turn['time'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.time'''